El siguiente código te dice, de una lista de skus por taller, si están disponibles en página o no

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
from datetime import date, timezone
import re
from tqdm import tqdm
import gspread
from sqlalchemy import create_engine
engine = create_engine('postgresql://eduardotobu:gbh3BCbjo*s@@127.0.0.1:9090/upsert')

from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

import gc
#desactivamos el garbage collector
gc.enable()

#today = date.today()
#titulos=[]
#precios = []
#ids = []
#ftitulos = []
#fprecios =[]
#fids=[]
#skusdobles=[]
#fskus=[]
#fpics=[]
#fotos=[]

In [2]:
sheet = client.open('SHOPPING X SKU 19 JUN - 5 JUL 2023').worksheet("Resumen pa' Edu")

df_trgt = pd.DataFrame(sheet.get_all_records(numericise_ignore=['all']))
df_trgt = df_trgt.rename(columns = {'Taller':'workshop_name','SKU (mayus)':'sku','Se Scrapea?':'scrap'})
df_trgt = df_trgt.loc[df_trgt['scrap']=='1']
df_trgt = df_trgt[['workshop_id','sku']]
df_trgt = df_trgt.astype({'workshop_id':int})
df_trgt

,workshop_id,sku
0,3,11839
1,499,16656
2,3,13687-P
3,100955,14851
4,3,3900-C
...,...,...
549,103455,10631
550,105186,10777
551,103455,16639
552,103455,12193


In [11]:
query = '''
SELECT *,
case 
	when workshop_id = 3 then 'https://www.enviaflores.com/florerias-nuevo-leon/monterrey/'
	when workshop_id = 499 then 'https://www.enviaflores.com/florerias-jalisco/zapopan/'
	when workshop_id = 678 then 'https://www.enviaflores.com/florerias-queretaro/queretaro/'
	when workshop_id = 1133 then 'https://www.enviaflores.com/florerias-puebla/puebla/'
	when workshop_id = 1168 then 'https://www.enviaflores.com/florerias-guanajuato/leon/'
	when workshop_id = 100874 then 'https://www.enviaflores.com/florerias-quintana-roo/cancun-benito-juarez/'
	when workshop_id = 100954 then 'https://www.enviaflores.com/florerias-san-luis-potosi/san-luis-potosi/'
	when workshop_id = 100955 then 'https://www.enviaflores.com/florerias-aguascalientes/aguascalientes/'
	when workshop_id = 102634 then 'https://www.enviaflores.com/florerias-estado-mexico/toluca/'
	when workshop_id = 102780 then 'https://www.enviaflores.com/florerias-coahuila/saltillo/'
	when workshop_id = 102941 then 'https://www.enviaflores.com/florerias-coahuila/torreon/'
	when workshop_id = 103298 then 'https://www.enviaflores.com/florerias-sonora/hermosillo/'
	when workshop_id = 103455 then 'https://www.enviaflores.com/florerias-distrito-federal-ciudad-mexico/benito-juarez/'
	when workshop_id = 104120 then 'https://www.enviaflores.com/florerias-yucatan/merida/'
	when workshop_id = 105185 then 'https://www.enviaflores.com/florerias-baja-california/tijuana/'
	when workshop_id = 105186 then 'https://www.enviaflores.com/florerias-chihuahua/chihuahua/'
	end as url_city
FROM(
	SELECT 
		*,   
		rank() over (partition by workshop_id order by venta desc) rn
		FROM
			(
			SELECT
				workshop_id,
				workshop_name,
				city_id,
				city_name,
				SUM(product_value) AS venta
			 
			FROM order_sales 
			WHERE
				 order_status_id = 3
				 and is_test_order = FALSE
				 AND is_sales_valid = TRUE
				 AND delivery_date BETWEEN CURRENT_DATE - INTERVAL'1 MONTH' AND CURRENT_DATE
				 AND workshop_id IS NOT NULL
			GROUP BY 1,2,3,4
			)
		)
WHERE rn = 1
'''
query = query.replace('\n',' ').replace('\t',' ')
df_city = pd.read_sql(query, engine)
df_city = df_city[['workshop_id','workshop_name','city_id','city_name','url_city']]
df_trgt_1 = df_trgt.merge(df_city, how = 'left', on = 'workshop_id')
df_trgt_1 = df_trgt_1[['workshop_id','workshop_name','city_id','city_name','url_city','sku']]
df_trgt_1['url_product'] = df_trgt_1['url_city'] + df_trgt_1['sku']
df_trgt_1

,workshop_id,workshop_name,city_id,city_name,url_city,sku,url_product
0,3,Monterrey,15696,Monterrey,https://www.enviaflores.com/florerias-nuevo-le...,11839,https://www.enviaflores.com/florerias-nuevo-le...
1,499,Guadalajara,15360,Zapopan,https://www.enviaflores.com/florerias-jalisco/...,16656,https://www.enviaflores.com/florerias-jalisco/...
2,3,Monterrey,15696,Monterrey,https://www.enviaflores.com/florerias-nuevo-le...,13687-P,https://www.enviaflores.com/florerias-nuevo-le...
3,100955,Aguascalientes,14713,Aguascalientes,https://www.enviaflores.com/florerias-aguascal...,14851,https://www.enviaflores.com/florerias-aguascal...
4,3,Monterrey,15696,Monterrey,https://www.enviaflores.com/florerias-nuevo-le...,3900-C,https://www.enviaflores.com/florerias-nuevo-le...
...,...,...,...,...,...,...,...
549,103455,Valle de Mex,14980,Benito Juarez,https://www.enviaflores.com/florerias-distrito...,10631,https://www.enviaflores.com/florerias-distrito...
550,105186,Chihuahua,14924,Chihuahua,https://www.enviaflores.com/florerias-chihuahu...,10777,https://www.enviaflores.com/florerias-chihuahu...
551,103455,Valle de Mex,14980,Benito Juarez,https://www.enviaflores.com/florerias-distrito...,16639,https://www.enviaflores.com/florerias-distrito...
552,103455,Valle de Mex,14980,Benito Juarez,https://www.enviaflores.com/florerias-distrito...,12193,https://www.enviaflores.com/florerias-distrito...


In [17]:
sorry_indicator = soup.find('div', attrs = {'class':'d-flex flex-column flex-md-row justify-content-center align-items-center bg-primary text-white p-2 container-fluid'})

print(sorry_indicator)

None


In [27]:
prods_en_pagina = list()
for index,row in df_trgt_1.iterrows():
    page_to_scrape = requests.get(row['url_product'])
    soup = BeautifulSoup(page_to_scrape.text, 'html.parser')
    sorry_indicator = soup.find('div', attrs = {'class':'d-flex flex-column flex-md-row justify-content-center align-items-center bg-primary text-white p-2 container-fluid'})#.findAll('a', attrs = {'class':'unstyle'}, href= True)
    if sorry_indicator == None:
        row['en_pagina'] = 1
    else:
        row['en_pagina'] = 0
    row = {"workshop_id":row['workshop_id'],"workshop_name":row['workshop_name'],"sku":row['sku'],"url":row['url_product'],'en_pagina':row['en_pagina']}
    print(row)
    prods_en_pagina.append(row)
    
df_prods_en_pagina = pd.DataFrame(prods_en_pagina)
df_prods_en_pagina.to_clipboard()

{'workshop_id': 3, 'workshop_name': 'Monterrey', 'sku': '11839', 'url': 'https://www.enviaflores.com/florerias-nuevo-leon/monterrey/11839', 'en_pagina': 1}
{'workshop_id': 499, 'workshop_name': 'Guadalajara', 'sku': '16656', 'url': 'https://www.enviaflores.com/florerias-jalisco/zapopan/16656', 'en_pagina': 0}
{'workshop_id': 3, 'workshop_name': 'Monterrey', 'sku': '13687-P', 'url': 'https://www.enviaflores.com/florerias-nuevo-leon/monterrey/13687-P', 'en_pagina': 0}
{'workshop_id': 100955, 'workshop_name': 'Aguascalientes', 'sku': '14851', 'url': 'https://www.enviaflores.com/florerias-aguascalientes/aguascalientes/14851', 'en_pagina': 1}
{'workshop_id': 3, 'workshop_name': 'Monterrey', 'sku': '3900-C', 'url': 'https://www.enviaflores.com/florerias-nuevo-leon/monterrey/3900-C', 'en_pagina': 1}
{'workshop_id': 103455, 'workshop_name': 'Valle de Mex', 'sku': '2562', 'url': 'https://www.enviaflores.com/florerias-distrito-federal-ciudad-mexico/benito-juarez/2562', 'en_pagina': 1}
{'workshop

NameError: name 'df_data_productos' is not defined

In [28]:
df_prods_en_pagina.to_clipboard()


In [4]:
query = '''
SELECT 
codigo as sku,
name as product_name,
categoria AS product_category_name,
subcategoria as product_subcategory_name,
linea as product_line_name
FROM bi_products
'''
query = query.replace('\n',' ').replace('\t',' ')
df_productos = pd.read_sql(query, engine)
df_trgt_2 = df_trgt_1.merge(df_productos, how = 'left', on = 'sku')
df_trgt_2
#df_trgt_1 = df_trgt_1[['workshop_id','workshop_name','city_id','city_name','sku']]
#df_trgt_1

,workshop_id,workshop_name,city_id,city_name,sku,product_name,product_category_name,product_subcategory_name,product_line_name
0,3,Monterrey,15696,Monterrey,11839,"Globo Esfera Magenta ""Happy Birthday"" con Muff...",Postres,Reposteria Fina,Muffins/Pasteles Ind
1,499,Guadalajara,15360,Zapopan,16656,Planta tu intenciÃ³n ''Nuevos comienzos'',Plantas,Semillas,Semillas
2,3,Monterrey,15696,Monterrey,13687-P,Tabla Personalizada 30x40 cm Texto Inferior De...,Grill,Tabla,Personalizado
3,100955,Aguascalientes,14713,Aguascalientes,14851,Globo Love You y Vela Amor,Velas,Marca Propia,Velas
4,3,Monterrey,15696,Monterrey,3900-C,Cruz Condolencias,Flores,Funerales,Coronas y Cubre Caja Funerales
...,...,...,...,...,...,...,...,...,...
549,103455,Valle de Mex,14980,Benito Juarez,10631,Perfume Coach para Dama 100ml,Perfumes,Individual,Para Ella
550,105186,Chihuahua,14924,Chihuahua,10777,Diario de Gratitud: Cartas al Universo,Libros y Agendas,Libros,Libros
551,103455,Valle de Mex,14980,Benito Juarez,16639,Cartas Seminario Puntos Positivos,Libros y Agendas,Agendas,Agendas
552,103455,Valle de Mex,14980,Benito Juarez,12193,Arreglo de Globos con Figura de Birrete,Globos,Graduation,Figuras


In [6]:
%%time

page = 'https://www.enviaflores.com/florerias-nuevo-leon/monterrey'
#Definicion de tiendas.
page_to_scrape = requests.get(page)
soup = BeautifulSoup(page_to_scrape.text, 'html.parser')

products = soup.find('div', attrs = {'id':'productsGrid','class':'d-flex pt-1 pt-md-3 flex-wrap'}).findAll('a', attrs = {'class':'unstyle'}, href= True)
for product in products:
    url_product = "https://www.enviaflores.com" + product.get('href')
    print(url_product)
   

https://www.enviaflores.com/florerias-nuevo-leon/monterrey/6035
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/6035
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/14891
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/14891
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/12041
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/12041
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/16764-E
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/16764-E
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/16521-P
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/16521-P
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/6689
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/6689
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/11839
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/11839
https://www.enviaflores.com/florerias-nuevo-leon/monterrey/001
https://www.enviaflores